In [2]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import Ridge, Lasso
from math import sqrt
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV

In [3]:
def createdata(X):
    XS  = np.square(X)
    XE = np.exp(X)
    XC = np.cos(X)
    ones = pd.DataFrame(np.ones((700, 1)))
    NX = pd.concat([X,XS, XE, XC, ones], axis = 1)
    return NX

In [4]:
def dataprep(file):
    data = pd.read_csv(file, index_col= 'Id')
    y =  data['y']
    X = data.drop('y', axis = 1)
    return X, y 

In [5]:
X, y = dataprep('train.csv')

In [6]:
NX = createdata(X)

In [7]:
def KfoldRig(data, nfold, rs):
    kf = KFold(n_splits=nfold,shuffle=True, random_state=rs)
    lambdas = np.logspace(0, 2, 200)
    RMSE = []
    for l in lambdas:
        score = []
        for train_index, test_index in kf.split(data):
            rig_mod = Ridge(alpha=l, normalize=True)
            rig_mod.fit(X.iloc[train_index], y[train_index])
            y_pred= rig_mod.predict(X.loc[test_index])
            MSE = mean_squared_error(y[test_index], y_pred)
            score.append(MSE/nfold)
        RMSE.append(sqrt(sum(score)))
    return RMSE        

In [8]:
def minAlpha(RMSE):
    m = min(RMSE)
    return list(np.logspace(0, 4, 200))[[i for i, j in enumerate(RMSE) if j == m]]

In [9]:
RMSE =KfoldRig(NX,5,2)

In [12]:
ridge = Ridge(alpha=minAlpha(RMSE), fit_intercept=True)

In [294]:
pd.Series(ridge.coef_).to_csv("Attempt18.csv", index =False)